In [6]:
import os

In [3]:
javelin_api_key = ""
llm_api_key = ""

### How Javelin and CrewwAI Integrate

## Javelin Setup:

- **API Keys**: Javelin provides an API key (`x-api-key`) to authenticate the requests.
- **Javelin Route**: Defines the specific route to a task or flow.

## Calling CrewwAI via Javelin:

- **CrewwAI** utilizes Javelin’s routes to send and manage requests.
  
  - **Flow Definition**: CrewwAI uses the Javelin API to define agents (e.g., market analyst, strategist) and their corresponding tasks.
  
  - **Task Execution**: Each agent, when triggered via the route, will execute the specific task as per the flow defined in CrewwAI.
  
  - **Flow Management**: CrewwAI listens to each agent’s output and triggers the next step or task, ensuring smooth workflow orchestration.


### Flow Execution Order:

##### The tasks are now executed in sequence: research_task → project_understanding_task → marketing_strategy_task → campaign_idea_task → copy_creation_task

No Task Skipped:

All tasks are connected in a linear chain. The flow will not skip any task unless an error occurs or a condition is met that stops execution.
Using @listen() Decorators:

Each task is marked with @listen(), which means they will listen for the completion of the preceding task and then take the result as input for the next task.
Result:

The flow will execute all tasks, starting from the market research all the way to copy creation.
The final output, Final Marketing Copies, will be printed when the flow completes.

In [4]:
import yaml
import os
import asyncio
import nest_asyncio  # Allows nested async loops in Jupyter
from dotenv import load_dotenv
from openai import OpenAI
from crewai.flow.flow import Flow, listen, start
javelin_api_key = ""
llm_api_key = ""
# Apply nest_asyncio for Jupyter compatibility
nest_asyncio.apply()

# Load environment variables from .env file
load_dotenv()



if not javelin_api_key or not llm_api_key:
    raise ValueError("API keys not found. Ensure .env file is properly loaded.")

# Define Javelin Headers
javelin_headers = {
    "x-api-key": javelin_api_key,
    "x-javelin-route": "testing"  # Replace with correct Javelin route
}

# Javelin OpenAI Client
client = OpenAI(api_key=llm_api_key,
                base_url="https://api-dev.javelin.live/v1/query",
                default_headers=javelin_headers)

# Load YAML Configurations
def load_yaml(file_path):
    assert os.path.exists(file_path), f"{file_path} not found!"
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

agents_config = load_yaml("agents.yaml")
tasks_config = load_yaml("tasks.yaml")

class MarketingFlow(Flow):
    model = "gpt-3.5-turbo"

    @start()
    def research_task(self):
        """Conducts market research."""
        task = tasks_config["research_task"]
        prompt = task["description"].format(customer_domain="Tech Industry", project_description="AI-Powered CRM")

        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        research_report = response.choices[0].message.content
        self.state["research_report"] = research_report
        return research_report

    @listen(research_task)
    def project_understanding_task(self, research_report):
        """Understands the project details."""
        task = tasks_config["project_understanding_task"]
        prompt = task["description"].format(project_description="AI-Powered CRM")

        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        project_summary = response.choices[0].message.content
        self.state["project_summary"] = project_summary
        return project_summary

    @listen(project_understanding_task)
    def marketing_strategy_task(self, project_summary):
        """Develops marketing strategies."""
        task = tasks_config["marketing_strategy_task"]
        prompt = task["description"].format(customer_domain="Tech Industry", project_description="AI-Powered CRM")

        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        strategy_doc = response.choices[0].message.content
        self.state["marketing_strategy"] = strategy_doc
        return strategy_doc

    @listen(marketing_strategy_task)
    def campaign_idea_task(self, marketing_strategy):
        """Creates marketing campaign ideas."""
        task = tasks_config["campaign_idea_task"]
        prompt = task["description"].format(project_description="AI-Powered CRM")

        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        campaign_ideas = response.choices[0].message.content
        self.state["campaign_ideas"] = campaign_ideas
        return campaign_ideas

    @listen(campaign_idea_task)
    def copy_creation_task(self, campaign_ideas):
        """Writes marketing copies based on campaign ideas."""
        task = tasks_config["copy_creation_task"]
        prompt = task["description"].format(project_description="AI-Powered CRM")

        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        marketing_copies = response.choices[0].message.content
        self.state["marketing_copies"] = marketing_copies
        return marketing_copies

# Run Flow Asynchronously
async def run_flow():
    flow = MarketingFlow()
    result = await flow.kickoff_async()
    print(f"Final Marketing Copies: {result}")

await run_flow()


Final Marketing Copies: 1. Project Launch Campaign: MAKE WAY! The future is here, and it's bolder, brighter, and better than ever! Introducing the revolution of _______! Join us as we unveil the newest chapter of innovation. It's your turn to step into tomorrow TODAY!

2. Holiday Sale Campaign: Say goodbye to the old and hello to the bold! The most anticipated sale of the season is here. Unwrap the magic of our HUGE Holiday Sale – your winter wonderland of savings!

3. Fitness App Campaign: "Your life. Your pace. Your app." Break free from the chains of uninspiring workouts. Get in sync with the rhythm of your heart with our personalized Fitness App. Designed to keep the spring in your step, every step of the way!

4. Luxury Real Estate Campaign: Not just a house, welcome to your new lifestyle! Embrace the pinnacle of opulence and comfort in our luxurious homes. Experience an intimate illustration of grandeur –build memories that last a lifetime.

5. Environmental Awareness Campaign: "

Final Marketing Copies: 1. "Unleash Your Inner Chef with Our Culinary Adventure Campaign! Get ready to tantalize your taste buds and master new recipes with our cooking classes, chef demos, and hands-on workshops. Join us on a gastronomic journey that will inspire and delight your senses. Sign up now and elevate your culinary skills to the next level!"

2. "Transform Your Home with Our Design Makeover Campaign! Say goodbye to drab interiors and hello to chic, modern style. Let our expert designers guide you through the latest trends, tips, and tricks to revamp your space. From color consultations to furniture selection, we’ve got you covered. Get ready to be the envy of all your friends with a home makeover that will leave them speechless!"

3. "Discover Your Hidden Talent with Our Creative Arts Campaign! Unleash your creativity and explore your artistic side with our painting classes, sculpture workshops, and mixed media demonstrations. Whether you’re a beginner or a seasoned pro, our